
# Práctica: Normalización de Texto para Análisis Multilingüe en Reseñas de Productos

**Contexto:** Eres parte del equipo de NLP de **GlobalReviews**, una empresa que procesa reseñas de usuarios en español, francés e inglés para detectar defectos en productos electrónicos.



## Fase 1: Diagnóstico de Problemas

**Objetivo:** Identificar errores de codificación, acentos inconsistentes y contracciones.


In [ ]:

import pandas as pd

# Cargar dataset
df = pd.read_csv("resenas_multilingue.csv")
print(df.sample(5))  # Muestra algunos ejemplos

# Ejemplo crítico
ejemplo = "Ã‰ste celular es increÃ­ble!!! Pero tarda 5hs en cargarse... #BaterÃ­aMala 😞"
print(ejemplo)



## Fase 2: Normalización Unicode y Codificación

**Objetivo:** Corregir textos corruptos y unificar formatos Unicode.


In [ ]:

import chardet
import unicodedata

# Detección de codificación con chardet
with open("resenas_multilingue.csv", "rb") as f:
    print(chardet.detect(f.read(10000)))

# Ejemplo de corrección de codificación
def corregir_codificacion(texto):
    try:
        return texto.encode('latin1').decode('utf8')
    except:
        return texto

# Normalización Unicode
def normalizar_unicode(texto):
    return unicodedata.normalize('NFC', texto)

texto_corregido = corregir_codificacion(ejemplo)
texto_normalizado = normalizar_unicode(texto_corregido)
print(texto_normalizado)



## Fase 3: Manejo de Acentos y Contracciones

**Objetivo:** Eliminar acentos opcionales y expandir contracciones coloquiales.


In [ ]:

import re
from unidecode import unidecode

# Diccionario de contracciones
contracciones = {
    "q'huvo": "que hubo",
    "pq": "porque",
    " x ": " por "
}

# Normalización de hashtags
def normalizar_hashtag(hashtag):
    return "#" + unidecode(hashtag[1:]).lower()

# Pipeline completo de normalización
def normalizar_texto(texto):
    texto = corregir_codificacion(texto)
    texto = normalizar_unicode(texto)
    texto = texto.lower()
    for contra, exp in contracciones.items():
        texto = texto.replace(contra, exp)
    texto = re.sub(r'#\w+', lambda m: normalizar_hashtag(m.group()), texto)
    texto = unidecode(texto)
    texto = re.sub(r'\s+', ' ', texto)
    return texto.strip()

print(normalizar_texto("Ã‰ste es un buen móvil!!! No q'huvo fallos. #Duradero❤️"))



## Fase 4: Evaluación de Impacto

**Objetivo:** Medir cómo la normalización afecta la calidad del análisis.


In [ ]:

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Supongamos que df tiene columna 'texto' y 'etiqueta'
df['texto_normalizado'] = df['texto'].apply(normalizar_texto)
X_train, X_test, y_train, y_test = train_test_split(df['texto'], df['etiqueta'], test_size=0.3, random_state=42)
X_train_norm, X_test_norm = train_test_split(df['texto_normalizado'], test_size=0.3, random_state=42)

vectorizer = CountVectorizer()
Xv_train = vectorizer.fit_transform(X_train)
Xv_test = vectorizer.transform(X_test)

Xv_train_norm = vectorizer.fit_transform(X_train_norm)
Xv_test_norm = vectorizer.transform(X_test_norm)

model = LogisticRegression()
model.fit(Xv_train, y_train)
acc_orig = accuracy_score(y_test, model.predict(Xv_test))

model.fit(Xv_train_norm, y_train)
acc_norm = accuracy_score(y_test, model.predict(Xv_test_norm))

print("Accuracy sin normalización:", acc_orig)
print("Accuracy con normalización:", acc_norm)



## Entrega Final

1. Script: `normalizacion.py`
2. Diccionario contracciones: `contracciones.txt`
3. Reporte PDF comparativo
4. Casos de test: `test_normalizacion.json`
